In [ ]:
#@markdown # **STEP 1:**
#@markdown ## Requirements
from IPython.display import clear_output

print('[1;32mInstalling requirements...')
#%pip install --quiet blendmodes accelerate basicsr fonts font-roboto gfpgan gradio invisible-watermark numpy omegaconf opencv-contrib-python requests piexif Pillow pytorch-lightning==1.7.7 realesrgan scikit-image==0.19 timm==0.4.12 transformers==4.25.1 torch einops jsonmerge clean-fid resize-right torchdiffeq kornia lark inflection GitPython torchsde safetensors psutil rich
#%cd /content/
!git clone -q --branch main https://github.com/Ysb321/superlite
%cd /content/superlite
%pip install -r requirements.txt --quiet
!sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" /content/lite_colab/modules/sd_models.py
!sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' /content/lite_colab/startfk.py
!sed -i "s@map_location='cpu'@map_location='cuda'@" /content/colab/modules/extras.py
!cp /content/superlite/assets/blocks.py /usr/local/lib/python3.10/dist-packages/gradio/blocks.py

clear_output()
print('[1;32mDone!')
#!python launch.py --share --theme dark --api --ckpt-dir /content/models/ --disable-safe-unpickle --enable-insecure-extension-access --opt-sdp-attention --disable-console-progressbars --no-download-sd-model

In [ ]:
#@markdown # **STEP 1.1: Extensions install**

%cd /content/superlite/extensions-builtin/
!git clone https://github.com/YKefasu/sd-artists-ui-tweak sd-artists-ui-tweak
!git clone https://tinyurl.com/sd-controlnet sd-controlnet
!git clone https://tinyurl.com/infinite-image-browsing infinite-image-browsing
!git clone https://github.com/Iyashinouta/sd-model-downloader sd-model-downloader
clear_output()

%cd /content/superlite/extensions/
!git clone https://tinyurl.com/sd-huggingface huggingface
!git clone https://tinyurl.com/additional-networks additional-networks
!git clone https://github.com/fkunn1326/openpose-editor openpose-editor
!git clone https://tinyurl.com/depth-lib depth-lib
!git clone https://github.com/hnmr293/posex posex
!git clone https://tinyurl.com/3d-open-pose-editor 3d-open-pose-editor
!git clone https://tinyurl.com/sd-deforum sd-deforum
!git clone https://tinyurl.com/sd-two-shot two-shot
!git clone https://tinyurl.com/aspect-ratio-helper aspect-ratio-helper
!git clone https://tinyurl.com/asymmetric-tiling asymmetric-tiling
!git clone https://tinyurl.com/tagcomplete tagcomplete
!git clone https://tinyurl.com/sd-SAG sd_SAG
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111 ultimate-upscale-for-automatic1111
!git clone https://github.com/richrobber2/canvas-zoom canvas-zoom
!git clone https://github.com/Tsukreya/Umi-AI-debloat Umi-AI
!git clone https://github.com/Bing-su/adetailer adetailer
!git clone https://github.com/CiaraStrawberry/TemporalKit TemporalKit
!git clone https://github.com/Extraltodeus/depthmap2mask depthmap2mask
clear_output()

%cd /content/superlite/repositories
!git clone https://github.com/isl-org/MiDaS MiDaS

# Clear the output to keep the notebook clean
clear_output()

# Print the success message
print("✓ Done", "success")

In [ ]:
#@markdown # **STEP 2**
#@markdown ## Choose your models
#@markdown _Enter the links and corresponding safetensors parameters_

MODEL_LINK_1 = "https://huggingface.co/SG161222/Realistic_Vision_V2.0/resolve/main/Realistic_Vision_V2.0-fp16-no-ema-inpainting.safetensors" #@param {type:"string"}
MODEL_SAFETENSORS_1 = True #@param {type:"boolean"}

MODEL_LINK_2 = "https://huggingface.co/XpucT/Deliberate/resolve/main/Deliberate_v2.safetensors" #@param {type:"string"}
MODEL_SAFETENSORS_2 = True #@param {type:"boolean"}

MODEL_LINK_3 = "-" #@param {type:"string"}
MODEL_SAFETENSORS_3 = True #@param {type:"boolean"}

MODEL_LINK_4 = "-" #@param {type:"string"}
MODEL_SAFETENSORS_4 = False #@param {type:"boolean"}

MODEL_LINK_5 = "-" #@param {type:"string"}
MODEL_SAFETENSORS_5 = True #@param {type:"boolean"}

MODEL_LINKS = [
    {
        'link': MODEL_LINK_1,
        'safetensors': MODEL_SAFETENSORS_1
    },
    {
        'link': MODEL_LINK_2,
        'safetensors': MODEL_SAFETENSORS_2
    },
    {
        'link': MODEL_LINK_3,
        'safetensors': MODEL_SAFETENSORS_3
    },
    {
        'link': MODEL_LINK_4,
        'safetensors': MODEL_SAFETENSORS_4
    },
    {
        'link': MODEL_LINK_5,
        'safetensors': MODEL_SAFETENSORS_5
    }
]

if all(model['link'] for model in MODEL_LINKS):
  pth = '/content/superlite/models/Stable-diffusion/'
  for i, model in enumerate(MODEL_LINKS):
    link = model['link']
    safetensors = model['safetensors']

    if safetensors:
        modelname = "model.safetensors"
    else:
        modelname = "model.ckpt"

    dwnld = pth + f"model{i+1}.{modelname.split('.')[1]}"
    print(f'[1;32mDownloading model {i+1}...')
    !gdown --fuzzy -O $dwnld "$link"
    print(f'[1;32mModel {i+1} downloaded!')

  print('[1;32mAll models downloaded successfully!')
else:
  print('[1;31mPlease provide links and corresponding safetensors parameters for all models!')

In [ ]:
#@markdown # **STEP 2.1** (optional)
#@markdown ## Choose Lora model





import os
import urllib.parse
import requests

Lora_model_link1 = 'https://civitai.com/api/download/models/62833' #@param {type:"string"}
Lora_model_link2 = 'https://civitai.com/api/download/models/63006' #@param {type:"string"}
Lora_model_link3 = 'https://civitai.com/api/download/models/10580' #@param {type:"string"}
Lora_model_link4 = 'https://civitai.com/api/download/models/7804' #@param {type:"string"}
Lora_model_link5 = 'https://civitai.com/api/download/models/9739' #@param {type:"string"}

links = [
   Lora_model_link1,
   Lora_model_link2,
   Lora_model_link3,
   Lora_model_link4,
   Lora_model_link5
]

pth = '/content/lite_colab/models/Lora/'

os.makedirs(pth, exist_ok=True)

for link in links:
    if not link:
        continue  # Skip empty links
    parsed_url = urllib.parse.urlparse(link)
    if not parsed_url.scheme or not parsed_url.netloc:
        print(f'Error: Invalid URL - {link}')
        continue  # Skip invalid URLs
    filename = os.path.join(pth, os.path.basename(parsed_url.path))  # Extract the filename from the URL
    filename_with_extension = f"{filename}.safetensors"  # Append .safetensors extension to the filename
    print(f'Downloading model from {link}...')
    try:
        response = requests.get(link, stream=True)
        response.raise_for_status()  # Raise an exception for non-2xx status codes
        # Check if the response headers indicate a valid file
        content_length = response.headers.get('content-length')
        if not content_length or int(content_length) == 0:
            print(f'Error downloading model. File not found or empty: {link}')
            continue  # Skip further processing
        with open(filename_with_extension, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        if os.path.exists(filename_with_extension) and os.path.getsize(filename_with_extension) > 0:
            print('Download complete!')
        else:
            if os.path.exists(filename_with_extension):
                os.remove(filename_with_extension)
            print(f'Error downloading model. File not found or empty: {link}')
    except requests.exceptions.RequestException as e:
        print(f'Error downloading model: {str(e)}')

In [ ]:
#@markdown # **STEP 3**
#@markdown ## Run StableDiffusion

!python launch.py --share --api --disable-safe-unpickle --enable-insecure-extension-access --opt-sdp-attention --disable-console-progressbars --no-download-sd-model